# 1.2 스크래핑
---

- **정규 표현식**을 이용한 스크래핑은 HTML을 단순한 문자열로 취급하여 필요한 정보 추출
    - 마크업 되지 않은 웹페이지도 문자열의 특징을 파악하면 스크래핑 가능 <br>
    
- XML 파서를 이용한 스크래핑은 XML 태그를 분석(파싱)하여 필요한 정보 추출
    - 블로그 또는 뉴스사이트 정보를 전달하기 위한 RSS와 같이 많은 데이터가 XML 형태로 제공
    - XML 파서를 사용하면 정규 표현식보다 간단하고 효과적으로 필요한 정보 추출가능


- HTML을 스크래핑할 때는 HTML 전용 파서가 필요
    - 파이썬의 표준 모듈인 html.parser 모듈을 사용하면 HTML 파싱 가능
    - lxml등과 같은 라이브러리를 사용하여 HTML 파싱 필요

## 정규식이란?

- 특정 검색 패턴에 대한 하나 이상의 일치 항목을 검색 
- 검색된 텍스트로부터 정보를 추출하는데 매우 유용하게 사용가능한 표현식
- 유효성 검사에서 문자열 파싱 및 대체, 데이터를 다른 형식으로 변환 및 웹 스크래핑에 이르기까지 다양한 응용분야에서 활용

<img src="img/정규식 패턴.PNG" width="550px" height="300px"></img><br/>
<img src="img/정규패턴식1.PNG" width="550px" height="300px"></img><br/>

#### 2.5.1.2.python_re 실습


In [1]:
import re
from html import unescape

In [13]:
# 이전 절에서 다운로드한 파일을 열고 html이라는 변수에 저장
with open('data/full_book_list.html', encoding='utf-8') as f:
    html = f.read()

In [16]:
# re.findall() 메서드를 통해 도서 하나에 해당하는 HTML을 추출
# 정규식을 작성할 때 re.DOTALL 옵션을 주면 \n 문자와도 매치된다.

for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
    # 도서의 URL을 추출
    url = re.search(r'<a href="(.*?)">', partial_html).group(1)
    url = 'http://www.hanbit.co.kr' + url
    # 태그를 제거해서 도서의 제목을 추출
    title = re.sub(r'<.*?>', '', partial_html)
    title = unescape(title)
    print('url:', url)
    print('title:', title)
    print('---')
    partial_html


url: http://www.hanbit.co.krhttps://www.hanbit.co.kr/store/books/look.php?p_code=B7623190015
title: 최신 관리회계
---
url: http://www.hanbit.co.krhttps://www.hanbit.co.kr/store/books/look.php?p_code=B4300598719
title: 리눅스 입문자를 위한 명령어 사전
---
url: http://www.hanbit.co.krhttps://www.hanbit.co.kr/store/books/look.php?p_code=B9108907099
title: 파타고니아 이야기
---
url: http://www.hanbit.co.krhttps://www.hanbit.co.kr/store/books/look.php?p_code=B7015117381
title: 풀스택 서버리스 :  리액트, AWS, 그래프QL을 이용한 최신 애플리케이션 개발
---
url: http://www.hanbit.co.krhttps://www.hanbit.co.kr/store/books/look.php?p_code=B7448632539
title: 한 권으로 배우는 작고 예쁜 꽃자수
---
url: http://www.hanbit.co.krhttps://www.hanbit.co.kr/store/books/look.php?p_code=B3148701816
title: IT CookBook, 처음 만나는 회로이론(2판)
---
url: http://www.hanbit.co.krhttps://www.hanbit.co.kr/store/books/look.php?p_code=B2591303726
title: 안전필수 시스템 제어 설계
---
url: http://www.hanbit.co.krhttps://www.hanbit.co.kr/store/books/look.php?p_code=B7468885216
title: 러닝 리액트(2판)
---
url: http:

---

## XML을 이용한 스크래핑

- RSS란?
    - 뉴스나 블로그 등 업데이트가 빈번한 사이트에서 주로 사용하는 콘텐츠 표현 방식
    - RSS 서비스를 이용하면 업데이트된 정보를 찾기 위해 홈페이지를 일일이 방문하지 않아도 업데이트 될때마다 빠르고 편리하게 확인가능

#### 2.5.2.1. python_rss 실습

In [21]:
from xml.etree import ElementTree
import pandas as pd

In [12]:
# parse() 함수로 파일을 읽어 들이고 ElementTree 객체를 만듭니다.
tree = ElementTree.parse('data/rss.xml')

In [13]:
# getroot() 메서드로 XML의 루트 요소를 추출
root = tree.getroot()

In [28]:
# findall() 메서드로 XML의 루트 요소를 추출
# 태그를 찾습니다.
# 데이터 프레임에 저장
weather_list = []

# xml을 열어보고 문서의 구조를 따라가면서 찾고자 하는 데이터 루트를 findall에 작성
for item in root.findall('channel/item/description/body/location/data'):
    # find() 메서드로 요소를 찾고 text 속성으로 값을 추출
    tm_ef = item.find('tmEf').text
    tmn = item.find('tmn').text
    tmx = item.find('tmx').text
    wf = item.find('wf').text
    list = pd.DataFrame({
        '일시':[tm_ef],
        '최저기온':[tmn],
        '최고기온':[tmx],
        '날씨':[wf]
    })
    weather_list.append(list)

# 데이터프레임으로
weather_list = pd.concat(weather_list)
weather_list

,일시,최저기온,최고기온,날씨
0,2020-06-25 00:00,21,26,흐리고 비
0,2020-06-25 12:00,21,26,흐리고 비
0,2020-06-26 00:00,21,29,흐리고 비
0,2020-06-26 12:00,21,29,구름많음
0,2020-06-27 00:00,22,29,구름많음
...,...,...,...,...
0,2020-06-29 00:00,23,26,흐리고 비
0,2020-06-29 12:00,23,26,흐리고 비
0,2020-06-30 00:00,22,26,흐리고 비
0,2020-07-01 00:00,22,26,흐리고 비
